In [2]:
# conding: utf-8

## imports

In [3]:
import math
import numpy as np
import pandas as pd
import seaborn as sbn
import matplotlib.pyplot as plt

In [4]:
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import train_test_split

## functions

In [5]:
def sign(a):
    return -1 if a < 0 else 1

def r_score(model, x, y):
    """
        model: model fitted
        x: target labels
        y: target values
    """
    r2 = model.score(x, y)
    return sign(r2) * math.sqrt(abs(r2))

In [6]:
with pd.HDFStore("../data/train.h5", "r") as data:
    DATA = data.get("train")

In [14]:
#DATA_MEANS = DATA.mean()
DATA.fillna(0, inplace=True)
DATA.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
0,10,0,0.370326,-0.006316,0.222831,-0.213030,0.729277,-0.335633,0.113292,1.621238,...,0.775208,0.0,0.0,0.0,-0.414776,0.0,0.0,-2.0,0.0,-0.011753
1,11,0,0.014765,-0.038064,-0.017425,0.320652,-0.034134,0.004413,0.114285,-0.210185,...,0.025590,0.0,0.0,0.0,-0.273607,0.0,0.0,-2.0,0.0,-0.001240
2,12,0,-0.010622,-0.050577,3.379575,-0.157525,-0.068550,-0.155937,1.219439,-0.764516,...,0.151881,0.0,0.0,0.0,-0.175710,0.0,0.0,-2.0,0.0,-0.020940
3,25,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.178495,0.000000,-0.007262,...,1.035936,0.0,0.0,0.0,-0.211506,0.0,0.0,-2.0,0.0,-0.015959
4,26,0,0.176693,-0.025284,-0.057680,0.015100,0.180894,0.139445,-0.125687,-0.018707,...,0.630232,0.0,0.0,0.0,-0.001957,0.0,0.0,0.0,0.0,-0.007338


## decomposition

In [ ]:
N_PCA = 16
pca = PCA(n_components=N_PCA)
pca_series = pca.fit_transform(DATA.drop(["id", "timestamp", "y"], axis=1))

In [ ]:
pca_data = pd.DataFrame(pca_series)
pca_cols = ['pca_%i'%i for i in range(16)]
pca_data.columns = pca_cols
pca_data['y'] = DATA.y

In [37]:
pca_data.head()

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,y
0,7.025244e+13,-9.443947e+11,-7.690958e+11,-87345528.0,-83019224.0,-346602112.0,-405677344.0,-560804864.0,93713656.0,-217763856.0,144555376.0,-9.377616e+06,152421824.0,-319614624.0,-510856160.0,-3.983390e+06,-0.011753
1,6.765066e+13,-9.413881e+11,-7.769983e+11,-109680864.0,-505652288.0,-31164050.0,339087808.0,38756624.0,-345962976.0,-136709904.0,-48266204.0,-8.684914e+06,81958008.0,-77783856.0,76160728.0,-9.005951e+05,-0.001240
2,7.090139e+13,-9.426091e+11,-7.728511e+11,-305789056.0,-518950016.0,-116932120.0,167009328.0,35020836.0,-61218308.0,224831136.0,-13917292.0,-8.638020e+06,-6113353.0,-198659200.0,-198082544.0,-1.457946e+06,-0.020940
3,7.089455e+13,-7.750121e+09,1.805344e+08,-35244064.0,-288725088.0,18889320.0,-392872480.0,30961712.0,40370344.0,126052208.0,32557010.0,2.510962e+03,138579792.0,33967968.0,379891744.0,2.567451e+06,-0.015959
4,6.950582e+13,-9.685481e+11,-7.728469e+11,-520303584.0,-322812576.0,-12745655.0,273235360.0,-24428322.0,-222379888.0,-175087456.0,-77109856.0,-8.993822e+06,101012136.0,43213280.0,303975616.0,6.557054e+05,-0.007338


## Sampling

In [15]:
DATA_TRAIN, DATA_VALID = train_test_split(DATA, test_size=0.2, random_state=42)

In [16]:
features = ["technical_30", "fundamental_11", "technical_20"]

In [21]:
tree = DecisionTreeRegressor()

In [23]:
tree.fit(DATA_TRAIN.drop(['id', 'y', 'timestamp'], axis=1), DATA_TRAIN.y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [20]:
r_score(tree, DATA_VALID[features], DATA_VALID.y)

-0.9562045686561513